# Example: Run Python model using UQpy RunModel Class
Author: Michael D. Shields
Date: May 17, 2018

In this example, the component-wise modified Metropolis-Hastings is used to draw a small set of (nearly) independent two-dimensional samples from a Rosenbrock distribution. These samples are passed to a Python model (python_model.py) that simply evaluates the sum of the components of each sample and returns this sum as the quantity of interest.

Import the necessary packages

In [1]:
from UQpy.SampleMethods import MCMC
from UQpy.RunModel import RunModel
import numpy as np
import time

Define a Rosenbrock pdf (up to a scale factor) and draw samples from it using the component-wise Modified Metropolis Hanstings algorithm. For illustration purposes, we draw 15 samples with jump of 100,000 samples given the very long correlation length for the MMH on the Rosenbrock density. See Goodman and Weare, 2010.

In [2]:
def Rosenbrock(x, params):
    return np.exp(-(100*(x[1]-x[0]**2)**2+(1-x[0])**2)/params[0])

t = time.time()
x = MCMC(dimension=2, pdf_proposal_type='Normal', pdf_target_type='joint_pdf',
         pdf_target=Rosenbrock, pdf_target_params=[20], algorithm='MMH', jump=100000, nsamples=15, seed=None)
t_MCMC = time.time()-t
print(t_MCMC)

30.151395082473755


Run the model 'python_model.py' for each s. This model contains a single class that accepts samples from x (x.samples) and evaluates the sum of the components of each sample over the two dimensions.

In [4]:
t = time.time()
z = RunModel(cpu=1, model_type='python', model_script='python_model.py', dimension=2, samples=x.samples)
t_run = time.time()-t
print(t_run)


Evaluating the model...

0.0033211708068847656


Observe how the data is stored. Notice that the data is stored inside UQpy in a manner consistent with the way the data is stored if the model is run using third-party software either in parallel or serial.

In [5]:
print('Samples',z.model_eval.samples)
print('Soluations',z.model_eval.QOI)

Samples [[  0.           0.        ]
 [  1.46075482   1.95478602]
 [  0.05443252  -0.69119813]
 [  0.33036508   0.092916  ]
 [  0.25731735   0.38677857]
 [  4.65632673  21.59315043]
 [  1.25911346   1.69504015]
 [  2.10803731   3.90503472]
 [  0.72722666   0.62894236]
 [  0.39259136   0.07116775]
 [  0.67580745   0.28434175]
 [ -0.1190521   -0.39430051]
 [  2.10449142   4.75837251]
 [  1.72901917   2.37298021]
 [  6.25319072  38.83242113]]
Soluations [0.0, 3.4155408418590589, -0.63676560874264088, 0.42328108629629146, 0.64409591291744583, 26.249477165774387, 2.9541536103895614, 6.0130720306013297, 1.3561690143960232, 0.46375910739594972, 0.96014919987312553, -0.51335260861813237, 6.8628639270601326, 4.1019993784158801, 45.08561185468843]


Notice that the model does not create the 'UQpyOut' directory. All calculations are performed internally in python and are not saved to text files.